## DATA INGESTION TO VECTOR DB PIPELINE

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

/opt/anaconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")


##.  each page have a document 

Found 1 PDF files to process

Processing: data1.pdf
  ✓ Loaded 60 pages

Total documents loaded: 60


In [3]:
 ### Text splitting get into chunks

def split_documents(documents,chunk_size=100,chunk_overlap=20):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [4]:
chunks =split_documents(all_pdf_documents)

Split 60 documents into 1100 chunks

Example chunk:
Content: Section 1: Introduction and Overview...
Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-10-13T08:33:41+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-10-13T08:33:41+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '../data/pdf/data1.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1', 'source_file': 'data1.pdf', 'file_type': 'pdf'}


## EXECUTING AND vectorstoreDB



In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb. config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics. pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


##  embedding model is ready 

## just need to use it 

# vector store db for storing the vector in db

In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store (save in db )
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name, ## where to store in the vectordb
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [9]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts) ## list of strings

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 1100 texts...


Batches: 100%|██████████| 35/35 [00:04<00:00,  7.79it/s]


Generated embeddings with shape: (1100, 384)
Adding 1100 documents to vector store...
Successfully added 1100 documents to vector store
Total documents in collection: 1100


 # RAG RETRIEVER FROM THE VECTORSTORE 

In [10]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
     
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)


In [11]:
rag_retriever

In [12]:
rag_retriever.retrieve("student") ## we will get a context now

Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_9aea7f50_692',
  'content': 'Section 7: Student Features',
  'metadata': {'creationdate': '2025-10-13T08:33:41+00:00',
   'creator': '(unspecified)',
   'title': '(anonymous)',
   'total_pages': 60,
   'subject': '(unspecified)',
   'page_label': '39',
   'moddate': '2025-10-13T08:33:41+00:00',
   'source': '../data/pdf/data1.pdf',
   'trapped': '/False',
   'content_length': 27,
   'keywords': '',
   'page': 38,
   'file_type': 'pdf',
   'producer': 'ReportLab PDF Library - www.reportlab.com',
   'source_file': 'data1.pdf',
   'author': '(anonymous)',
   'doc_index': 692},
  'similarity_score': 0.16122019290924072,
  'distance': 0.8387798070907593,
  'rank': 1},
 {'id': 'doc_5fe1ce55_1022',
  'content': 'Section 10: Student Features',
  'metadata': {'subject': '(unspecified)',
   'moddate': '2025-10-13T08:33:41+00:00',
   'content_length': 28,
   'page_label': '57',
   'keywords': '',
   'file_type': 'pdf',
   'producer': 'ReportLab PDF Library - www.reportlab.com',
   't

## LLM integration. with the context pipeline

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize Groq LLM
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant",
    temperature=0.2,
    max_tokens=2048
)

def rag_portal_assistant(user_query, retriever, llm, top_k=5):
    """
    RAG pipeline for Student Management Portal chatbot.
    Retrieves relevant content from the portal guide and generates a contextual answer.
    """

    # Step 1: Retrieve top-k relevant guide chunks
    guide_results = retriever.retrieve(user_query, top_k=top_k)
    guide_context = "\n\n".join([doc['content'] for doc in guide_results]) if guide_results else ""

    if not guide_context:
        return "Sorry, I couldn’t find relevant information in the guide for your query."

    # Step 2: Construct contextual prompt
    prompt = f"""
You are a **Student Management Portal assistant** helping teachers and students.
Answer queries strictly based on the following guide context.

---
### 📘 Portal Guide Context:
{guide_context}

---
### 💬 User Query:
{user_query}

---

### 🧠 Instructions:
- Answer clearly and politely.
- If the question is related to portal usage, explain the exact steps.
- If it’s about academic or administrative policy, summarize the rule.
- If information is not in the guide, respond: *"This detail is not mentioned in the portal guide."*
- Avoid assumptions — stay factual.

---

### ✅ Response (Markdown Format):

**Answer:**
- ...

**Related Module:**
- ...

**Tip (if applicable):**
- ...
"""

    # Step 3: Generate response
    response = llm.invoke(prompt)
    return response.content


In [18]:
query = "Exam"
answer = rag_portal_assistant(query, rag_retriever, llm)
print(answer)

Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.78it/s]


Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)
**Answer:**
You can view exam schedules, grades, and performance analytics within the 'Exams and Results' module of the portal.

**Related Module:**
Exams and Results

**Tip (if applicable):**
To access the 'Exams and Results' module, please follow these steps:
1. Log in to the portal.
2. Click on the 'Exams and Results' tab on the top navigation menu.
3. Select the relevant exam schedule, grade, or performance analytics from the available options.


# ENHANCED RAG PIPELINE

In [20]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Exams query", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.25it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: Exam schedules, grades, and performance analytics are available within the 'Exams and Results'.
Sources: [{'source': 'data1.pdf', 'page': 26, 'score': 0.3047114610671997, 'preview': "Exam schedules, grades, and performance analytics are available within the 'Exams and Results'..."}, {'source': 'data1.pdf', 'page': 38, 'score': 0.3047114610671997, 'preview': "Exam schedules, grades, and performance analytics are available within the 'Exams and Results'..."}, {'source': 'data1.pdf', 'page': 56, 'score': 0.3047114610671997, 'preview': "Exam schedules, grades, and performance analytics are available within the 'Exams and Results'..."}]
Confidence: 0.3047114610671997
Context Preview: Exam schedules, grades, and performance analytics are available within the 'Exams and Results'

Exam schedules, grades, and performance analytics are available within the 'Exams and Results'

Exam schedules, grades, and performance analytics are available within the 'Exams and Results'


In [23]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Exams", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.71it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Exam schedules, grades, and performance analytics are available within the 'Exams and Resul

ts'

Exam schedules, grades, and performance analytics are available within the 'Exams and Results'

Exam schedules, grades, and performance analytics are available within the 'Exams and Results'

Question: Exams

Answer:

Final Answer: Exams and Results

Citations:
[1] data1.pdf (page 8)
[2] data1.pdf (page 20)
[3] data1.pdf (page 56)
Summary: However, you haven't provided an answer for me to summarize. Please provide the text you would like me to summarize, and I'll be happy to assist you.
History: {'question': 'Exams', 'answer': 'Exams and Results', 'sources': [{'source': 'data1.pdf', 'page': 8, 'score': 0.2397938370704651, 'preview': "Exam schedules, grades, and performance analytics are available within the 'Exams and Results'..."}, {'source': 'data1.pdf', 'page': 20, 'score': 0.2397938370704651, 'preview': "Exam schedules, grades, and performance analytics are available within the 'Exams and Results'..."}, {'source': 'data1.pdf', 'page': 56, 'score': 0.2397938370704651, 'preview'